In [ ]:
import re
import os

from tqdm import tqdm, tnrange
import json
import pandas as pd
import os
from collections import namedtuple


from nltk.corpus import wordnet as wn
import copy
import numpy as np

import tensorflow.keras as K

import utils
import parsers
import models

training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml'
gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt'

# training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml'
# gold_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.gold.key.txt'

training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml'
gold_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.gold.key.txt'
fine_senses_vocab_path = '../resources/semcor.vocab.WordNet.json'
input_vocab_path = '../resources/semcor.input.vocab.json'
input_antivocab_path = '../resources/semcor.leftout.vocab.json'
embedding_size = 32
batch_size = 64
LEARNING_RATE = 0.01
N_EPOCHS = 10
PADDING_SIZE = 50
print_model = False

In [ ]:
#loading dict
senses = utils.json_vocab_reader(fine_senses_vocab_path)
inputs, antivocab = utils.json_vocab_reader(input_vocab_path, input_antivocab_path)
output_vocab = utils.vocab_merge(senses, inputs)
reverse_output_vocab =  dict((v, k) for k, v in output_vocab.items())

K.backend.clear_session()

In [ ]:
dir_ = "../resources/WSD_Evaluation_Framework/Evaluation_Datasets"
eval_datasets = [i for i in os.listdir(dir_) if i.startswith("se")]
eval_datasets

In [ ]:
# training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/semeval2015.data.xml'


In [ ]:
# ##################
# # Model loading #
# #################
loaded_model = K.models.load_model('../resources/models/model_2019-09-06_22:48:47_+0200.h5')
loaded_model.load_weights('../resources/models/model_weights_2019-09-06_22:48:47_+0200.h5')


In [6]:
# loaded_model.compile(loss = 'categorical_crossentropy',
#               optimizer = K.optimizers.Adam(lr=0.0015, clipnorm=1., clipvalue=0.5),
#               metrics = ['acc', K.metrics.Precision()])

# ## PREDICTION
# predicted_one_hot = loaded_model.predict([padded(X_test_uni), padded(X_test_bi)])
# #de - one hot encode
# predicted = np.argmax(predicted_one_hot, axis=2)

In [7]:
import generatorPrototype
import generators

In [8]:
def eval_parser(path = training_file_path_dev, batch_size = 64):

    data_flow = parsers.TrainingParser(path)
    sentence_batch = []
    for batch_count, sentence in enumerate(data_flow.parse(), start = 1):
        sentence_batch.append(sentence)

        if int(batch_count)%int(batch_size)==0:
            yield sentence_batch
            sentence_batch = []

In [9]:
batch_size = 64

In [10]:
eval_generator = generatorPrototype.get(batch_size = batch_size,
                                training_file_path = training_file_path_dev,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)

In [11]:
real_words = utils.eval_parser(path = training_file_path_dev, batch_size = batch_size)

for batch_x, candidate_synsets in eval_generator:
    batch_real_words = next(real_words)
    predictions = loaded_model.predict_on_batch(batch_x)
    break

In [23]:
class MFS(object):
    """
    WSD performed via Most Frequent Sense (MFS) which always returns the predominant sense of a lemma
    """

    def retrieve_item(lemma):
        """
        Retreives the MFS of a lemma from the nltk package if exists else word
        """
        synsets = wn.synsets(str(lemma))
        
        if len(synsets) == 0:
            print("fail")
            return lemma
        else:
            mfs = synsets[0]
            return utils.WordNet.from_synset(mfs)

    def predict(sentence):
        """
        param: sentence of lemmas
        return sentence MFS
        """
        return [MFS.retrieve_item(lemma) for lemma in sentence]

In [31]:
synsets = wn.synsets("to")
synsets

[]

In [24]:
for idx_sentence, sentence in enumerate(predictions):
    
    real_sentence = batch_real_words[idx_sentence]
    for idx, entry in enumerate(real_sentence):
        
        word_prob = sentence[idx]
        
        current_candidate_synsets = candidate_synsets[idx_sentence][idx]
        prob_dist_candidate_synset = word_prob[current_candidate_synsets]
        current_synset = np.argmax(prob_dist_candidate_synset)
#         print(current_synset)
        #TO DO:
        if current_synset>4:#change after deleting start stop
            single_lemma_prediction = reverse_output_vocab[current_synset]
            print(single_lemma_prediction)
        else:
            
            word = entry.lemma
            print(MFS.retrieve_item(word))

fail
u.n.
wn:00031264n
wn:13377268n
wn:05898568n
fail
to
wn:00429060v
wn:01253060n
fail
the
fail
u.n.-sponsored
wn:14519366n
wn:08308497n
fail
--
wn:00956687v
wn:06868844n
wn:08016900n
wn:00417787r
wn:01666489s
wn:04887701n
fail
and
wn:02152473s
wn:07234657n
fail
--
wn:13754293n
wn:01631072v
wn:02604760v
wn:15164463n
fail
with
fail
the
wn:02604760v
fail
of
wn:13658027n
wn:06470073n
wn:08612786n
wn:00104051a
fail
greenhouse-gas
wn:00351638n
wn:15258694n
fail
the
wn:00127948s
wn:13749527n
wn:15203791n
fail
,
fail
with
wn:00576979v
wn:08168978n
wn:05548840n
wn:01557120a
fail
of
fail
the
wn:05832745n
wn:13649791n
fail
the
wn:02053941v
wn:01437963a
fail
.
fail
the
wn:06387980n
fail
,
fail
which
fail
could
wn:02327200v
fail
the
wn:13790912n
fail
for
wn:13658027n
wn:07466832n
wn:01814385a
wn:01110274n
fail
to
wn:00299341v
wn:14877234n
fail
,
wn:13810141n
fail
the
wn:00117985v
wn:05690269n
wn:14629149n
wn:13776621n
wn:14629149n
wn:06134510n
wn:00291873v
wn:13658027n
wn:00415676n
wn:10105733n
f

IndexError: index 50 is out of bounds for axis 0 with size 50

In [ ]:
current_synset

In [ ]:
sentence[candidate_synsets[idx_batch][idx_sentence]]

In [ ]:
pred = x[0][0]
cs = candidate_synsets[0][0]
pred[cs]

In [ ]:
depth = lambda L: isinstance(L, list) and max(map(depth, L))+1

In [ ]:
depth(candidate_synsets)

In [ ]:
x[0, 0, :]

In [ ]:
len(batch)

In [ ]:
for sentence in candidate_synsets:
    for cs in sentence:
        print(cs)
        #break
    break

In [ ]:
cs = [1,3]

In [ ]:
pred[cs]

In [ ]:
pred[3]

In [ ]:
x[0][0]

In [ ]:
[reverse_output_vocab[i] for i in cs ]